# My E-Commerce Recommender

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

In [2]:
# Identity
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Specify an S3 Bucket and Data Output Location

Amazon Personalize will need an S3 bucket to act as the source of your data. The code bellow will create a bucket with a unique `bucket_name`.

The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources. 

In [4]:
s3 = boto3.client('s3')
# bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
bucket_name = "personalize-demo-ecom-" + account_id
print('bucket_name:', bucket_name)

try:
    s3.create_bucket(
        Bucket = bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
        )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

bucket_name: personalize-demo-ecom-993936030494
Bucket already exists. Using bucket personalize-demo-ecom-993936030494


In [5]:
# !aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .
# !aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

## Choose a recommender use cases

Each domain has different use cases. When you create a recommender you create it for a specific use case, and each use case has different requirements for getting recommendations.


In [6]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain.
display (available_recipes['recipes'])

[{'name': 'aws-ecomm-customers-who-viewed-x-also-viewed',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 10, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2023, 10, 27, 1, 11, 18, 356000, tzinfo=tzlocal()),
  'domain': 'ECOMMERCE'},
 {'name': 'aws-ecomm-frequently-bought-together',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 10, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2023, 10, 27, 1, 11, 18, 356000, tzinfo=tzlocal()),
  'domain': 'ECOMMERCE'},
 {'name': 'aws-ecomm-popular-items-by-purchases',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 10, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.dat

We are going to create a recommender of the type "Customers who viewed X also viewed". This recommender gives recommendations for items that customers also viewed based on an item that you specify. With this use case, Amazon Personalize automatically filters items the user purchased based on the userId that you specify and `Purchase` events.

In [7]:
# create_recommender_response = personalize.create_recommender(
#   name = 'viewed_x_also_viewed_demo',
#   recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
#   datasetGroupArn = dataset_group_arn
# )
create_recommender_response = {
  "recommenderArn": "arn:aws:personalize:ap-southeast-2:993936030494:recommender/viewed_x_also_viewed_demo",
  "ResponseMetadata": {
    "RequestId": "458486dd-ca54-46da-a847-3491790f2a81",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 05:10:30 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "106",
      "connection": "keep-alive",
      "x-amzn-requestid": "458486dd-ca54-46da-a847-3491790f2a81"
    },
    "RetryAttempts": 0
  }
}
viewed_x_also_viewed_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response, indent=2))

{
  "recommenderArn": "arn:aws:personalize:ap-southeast-2:993936030494:recommender/viewed_x_also_viewed_demo",
  "ResponseMetadata": {
    "RequestId": "458486dd-ca54-46da-a847-3491790f2a81",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 05:10:30 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "106",
      "connection": "keep-alive",
      "x-amzn-requestid": "458486dd-ca54-46da-a847-3491790f2a81"
    },
    "RetryAttempts": 0
  }
}


We are going to create a second recommender of the type "Recommended For You". This type of recommender offers personalized recommendations for items based on a user that you specify. With this use case, Amazon Personalize automatically filters items the user purchased based on the userId that you specify and `Purchase` events.

[More use cases per domain](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [8]:
# create_recommender_response = personalize.create_recommender(
#   name = 'recommended_for_you_demo',
#   recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
#   datasetGroupArn = dataset_group_arn
# )
create_recommender_response = {
  "recommenderArn": "arn:aws:personalize:ap-southeast-2:993936030494:recommender/recommended_for_you_demo",
  "ResponseMetadata": {
    "RequestId": "eea0e7df-b76e-403c-bac1-1efb0017baa1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 05:10:52 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "105",
      "connection": "keep-alive",
      "x-amzn-requestid": "eea0e7df-b76e-403c-bac1-1efb0017baa1"
    },
    "RetryAttempts": 0
  }
}
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response, indent=2))

{
  "recommenderArn": "arn:aws:personalize:ap-southeast-2:993936030494:recommender/recommended_for_you_demo",
  "ResponseMetadata": {
    "RequestId": "eea0e7df-b76e-403c-bac1-1efb0017baa1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 05:10:52 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "105",
      "connection": "keep-alive",
      "x-amzn-requestid": "eea0e7df-b76e-403c-bac1-1efb0017baa1"
    },
    "RetryAttempts": 0
  }
}


## Getting recommendations with a recommender
Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

In [9]:
# reading the original data in order to have a dataframe that has both item_ids
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

,ITEM_ID,PRICE,CATEGORY_L1,CATEGORY_L2,PRODUCT_DESCRIPTION,GENDER,PROMOTED
2458,24c62ad2-6977-4f69-be75-e37d897c1434,2.10,food service,soup and salad,Warms you to your bowels and delivers vitamins...,Any,N
80,9c05f815-7ecf-4054-9132-7f47455d3a43,25.99,accessories,belt,Swell belt for women,F,Y
1477,6d665e0d-486e-492b-9486-ba278f387f73,44.99,homedecor,decorative,This rattan basket is a must-have for your hom...,Any,N
2284,1d183b2d-09f0-409c-b1fe-8f91059654c9,28.99,seasonal,halloween,A must-have for that fanciful day loved by all,Any,N
87,b105252a-e06c-413a-a635-b911fc3c4033,46.99,accessories,belt,Sassy belt for women,F,N
1513,43c778d9-3e6c-4dec-aa34-e171b1f6c3c6,38.99,homedecor,decorative,This ceramic vase will delight everyone,Any,N
321,f51fb444-2d3a-4f88-bda3-667114177172,113.99,apparel,scarf,Sublime scarf for women,F,N
1449,d8bed77a-c906-4f06-aeb1-04c3557f2e8e,32.99,homedecor,cushion,This square cushion is a must-have for your house,Any,N
1786,1495081d-3fba-4cf0-a27d-afbae683f1b1,71.99,housewares,kitchen,A must-have for your kitchen,Any,Y
1272,3330b1e7-402c-40d7-8bf3-0a5248013381,5.99,groceries,fruits,Make sure to have your fresh dose of vitamin C...,Any,N


In [10]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.

    A really broad try/except clause was added in case anything goes wrong.

    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

Let us get some recommendations using the "Customers who viewed X also viewed" Recommender:

In [11]:
# use a random valid id for a quick sanity check, modify the line of code bellow to a valid id in your dataset
get_item_by_id("c72257d4-430b-4eb7-9de3-28396e593381", items_df)

'Your dog will love this accessory'

In [12]:
# First pick a user
test_user_id = "888"

# Select a random item
test_item_id = "8fbe091c-f73c-4727-8fe7-d27eabd17bea" # a random item: 8fbe091c-f73c-4727-8fe7-d27eabd17bea

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = test_item_id,
    userId = test_user_id,
    numResults = 10
)
print(json.dumps(get_recommendations_response, indent=2))

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_item_by_id(test_item_id, items_df)])

pd.options.display.max_rows =10
display(user_recommendations_df)

{
  "ResponseMetadata": {
    "RequestId": "cd5cb9da-b3a6-484f-b7d2-63b02cbb85e9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 23:54:13 GMT",
      "content-type": "application/json",
      "content-length": "1266",
      "connection": "keep-alive",
      "x-amzn-requestid": "cd5cb9da-b3a6-484f-b7d2-63b02cbb85e9"
    },
    "RetryAttempts": 0
  },
  "itemList": [
    {
      "itemId": "b803aff7-6e4b-4beb-8d1d-dc7fe609274d"
    },
    {
      "itemId": "dd9e97a6-4281-40a5-9048-fde7809623fb"
    },
    {
      "itemId": "75e9f144-28cd-4f72-af5d-63be5e3761c8"
    },
    {
      "itemId": "536b9388-bafb-432d-9085-ad4af01edd6f"
    },
    {
      "itemId": "34ef7f57-ed89-4708-809b-fe6ba43c5af0"
    },
    {
      "itemId": "6bc496d1-f431-4642-b9ae-9dcc5881afe9"
    },
    {
      "itemId": "7e0e0e1f-9798-4f20-843f-df9a1ae2d878"
    },
    {
      "itemId": "d61d81c7-dc92-463f-9f0b-3edfbe9516af"
    },
    {
      "itemId": "51de6572-fb70-4e41-8150-c107fd4

,This video camera is perfect for capturing those special moments
0,Laptop computer for your computing needs
1,This high quality cabling is unequaled for you...
2,Video camera for amateurs and professonals
3,Video camera for amateurs and professonals
4,Groovy pair of sandals for men
5,This desktop computer is a boon for productivity
6,Pair of crimson formal shoes for women
7,This wall clock will keep you on time
8,This high fidelity pair of headphones is unriv...
9,This dandyish pair of sandals for men is perfe...


Get recommendations from the recommender returning "Recommended for you":

In [13]:
# First pick a user
test_user_id = "888"

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)
print(json.dumps(get_recommendations_response, indent=2))

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)


user_recommendations_df = pd.DataFrame(recommendation_list, columns = [test_user_id])

pd.options.display.max_rows =20
display(user_recommendations_df)

{
  "ResponseMetadata": {
    "RequestId": "f7cd49ed-630e-41fa-b9f2-88adf0f3862f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 09 Nov 2023 23:54:13 GMT",
      "content-type": "application/json",
      "content-length": "2456",
      "connection": "keep-alive",
      "x-amzn-requestid": "f7cd49ed-630e-41fa-b9f2-88adf0f3862f"
    },
    "RetryAttempts": 0
  },
  "itemList": [
    {
      "itemId": "09920b2e-4e07-41f7-aca6-47744777a2a7"
    },
    {
      "itemId": "d9d3351f-1fdb-4ba7-b757-55f18a1a2dc0"
    },
    {
      "itemId": "6928c229-e860-45f9-8720-45e2ea2fae2f"
    },
    {
      "itemId": "73d9a14d-0fa0-44ca-b367-e4f9af3986ed"
    },
    {
      "itemId": "94a0ad41-8b19-4ecb-b0d7-33704e2d4421"
    },
    {
      "itemId": "5a9e66ed-32c5-461e-a4b0-a56948c3235b"
    },
    {
      "itemId": "1d3ae532-f790-44ca-a8e8-f55aa9b66526"
    },
    {
      "itemId": "6f5b874d-68c7-435d-a66d-8296461c10e4"
    },
    {
      "itemId": "f995ec8d-237c-4513-8bfa-9aee210

,888
0,A must-have in every bathroom
1,White backpack is nifty for traveling
2,This ultrahip watch for women is perfect for a...
3,Black backpack is useful for traveling
4,This canvas leather bag for women is outstandi...
5,An all-around convenient bag for everyday use
6,This purple backpack for women is flawless for...
7,This casual pale gray handbag is outstanding a...
8,This dark olive green backpack for women is ou...
9,Soft brush for everyday use


## End of page